In [1]:
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 0

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.set_num_threads(8)

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "CIFAR10",
            # 'dataset': "FMNIST",
            'random_class_idx' : False,
            'method': "",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 50,
            'per_task_examples': np.inf,
            # 'per_task_examples': 200000,
            'per_task_memory_examples': 64,
            'batch_size_train': 256,
            'batch_size_memory': 256,
            'batch_size_validation': 256,
            'tau': 2.0,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.01,
            'learning_rate_decay_epoch': [30, 50, 70, 90],
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:4' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.001,
            'lambda': 1.,
            'lambda_old': 0.0,
              }


#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=CIFAR10/seed=0_epoch=50_lr=0.01_alpha=0.001_tau=2


In [2]:
from datasets import MNIST, FashionMNIST, BiasedMNIST, CIFAR10, CIFAR100
if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        joint = (params['method'] == "joint"),
                        random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'FashionMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = params['per_task_examples'],
                                joint = (params['method'] == "joint"),
                                random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'CIFAR10':
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        joint = (params['method'] == "joint"),
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] == 'CIFAR100':        
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            joint = (params['method'] == "joint"),
                            random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            joint = (params['method'] == "joint"),
                            random_class_idx = params['random_class_idx'])
    input_dim = (3, 28, 28)
else:
    raise NotImplementedError
class_idx = benchmark.class_idx
num_classes = len(class_idx)


[0 1 2 3 4 5 6 7 8 9]
Files already downloaded and verified
Files already downloaded and verified


In [3]:
benchmark.trains[1].true_index

array([    4,     5,    29, ..., 49994, 49998, 49999])

In [4]:
torch.get_num_threads()

8

In [5]:
idx = 8

In [6]:
from algorithms.imbalance import Heuristic2
from metrics import MetricCollector2
from backbones import MLP2Layers2

from backbones.resnet import ResNet18


# backbone = ResNet18(
#     input_dim=input_dim, 
#     output_dim=num_classes,
#     class_idx=class_idx,
#     config=params
#     ).to(params['device'])

backbone = ResNet18(
    input_dim=input_dim, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])

algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

modify resnet for cifar


In [7]:
from trainers import ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [8]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 0.6295, 'loss': 0.003189984768629074, 'std': 0.31849999999999995, 'EER': -1}


In [ ]:
metric_manager_callback.meters['EER'].get_eer()

[0.0,
 0.0037500000000000033,
 0.012888888888888894,
 0.20231249999999998,
 0.21972000000000005]

In [ ]:
metric_manager_callback.meters['std'].get_std()

[0.04199999999999998,
 0.16236898564689012,
 0.1634619391649186,
 0.2486261940645032,
 0.2794584226678452]

In [ ]:
metric_manager_callback.meters['accuracy'].get_data()

array([[0.943, 0.   , 0.   , 0.   , 0.   ],
       [0.342, 0.335, 0.   , 0.   , 0.   ],
       [0.39 , 0.368, 0.402, 0.   , 0.   ],
       [0.222, 0.208, 0.172, 0.74 , 0.   ],
       [0.089, 0.132, 0.105, 0.176, 0.812]])

In [ ]:
metric_manager_callback.meters['accuracy'].compute_overall()

[0.9430000000000001,
 0.33875,
 0.3868333333333333,
 0.335375,
 0.26270000000000004]

In [ ]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"EER:{np.mean(metric_manager_callback.meters['EER'].compute_overall())}")
print(f"std:{np.mean(metric_manager_callback.meters['std'].compute_overall())}")


accuracy:0.4533316666666667
EER:0.08773427777777779
std:0.1791831083088314
